In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flask
!pip install pyngrok
!pip install flask-cors
!ngrok authtoken 2dQz9vgx2sRTgxs4fSymM2ANuoF_2K6xy4APgChmC4NtStL8H


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install transformers

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Read and preprocess the text files (same as before)
def preprocess_text(text):
    # Implement your text preprocessing steps here
    # For example, remove stopwords, punctuation, etc.
    return text

def read_text_files(directory):
    texts = []
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            with open(os.path.join(directory, filename), "r", encoding="utf-8") as file:
                text = file.read()
                # print("file read ",filename)
                preprocessed_text = preprocess_text(text)
                texts.append(preprocessed_text)
    return texts

# Tokenization and TF-IDF calculation (same as before)
def extract_keywords(texts):
    vectorizer = TfidfVectorizer(max_features=10000000)  # You can adjust max_features as needed
    tfidf_matrix = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()
    return tfidf_matrix, feature_names

# Extract keywords and assign labels (same as before)
def get_keywords_for_files(directory):
    texts = read_text_files(directory)
    tfidf_matrix, feature_names = extract_keywords(texts)

    keywords_per_file = {}
    for i, text in enumerate(texts):
        tfidf_scores = tfidf_matrix[i].toarray().flatten()
        keywords_indices = tfidf_scores.argsort()[-100000000000:]  # You can adjust the number of keywords to extract
        keywords = [feature_names[idx] for idx in keywords_indices]
        keywords_per_file[os.listdir(directory)[i]] = keywords

    return keywords_per_file

# Matching function between question and filenames based on keywords
def find_related_files(question, keywords_per_file,directory):
    question_keywords = preprocess_text(question).split()  # Preprocess the question and extract keywords

    related_files = []
    comapre_txt=""
    file_append=0
    for filename, keywords in keywords_per_file.items():
        if file_append > 0:
          continue
        intersection = set(keywords) & set(question_keywords)
        # print(intersection)
        # print(len(intersection))
        if len(intersection) > 2:
            file_append = file_append +1
            related_files.append(filename)
            with open(f'{directory}/{filename}', 'r') as file:
              comapre_txt += file.read()


    return {"related_files" : related_files , "txt" : comapre_txt}


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

def t5_question_answering(context, question):
    # Load pre-trained T5 model and tokenizer
    model_name = "t5-base"
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Prepare input
    input_text = "question: {} context: {}".format(question, context)
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Generate answer
    output = model.generate(input_ids=input_ids, max_length=50, num_return_sequences=1, early_stopping=True)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    return answer

In [ ]:
directory = "/content/drive/My Drive/content-chatbot-txt-files/"
keywords_per_file = get_keywords_for_files(directory)
question ="body municipal board n paragraphs 30-48meeting date ?"
related_files = find_related_files(question, keywords_per_file , directory)
print("Related files:", related_files)

content = related_files["txt"]
answer = t5_question_answering(content, question)
print("Ref Files :" , related_files["related_files"])
print("Answer:", answer)

Related files: {'related_files': ['1.txt'], 'txt': 'protokoll kommunfullmäktige 2024-02-29 signatur närvarande beslutande ledamöter péter kovács (m) (ordförande) ulf molin (c) (vice ordförande) wivi-anne broberg (s) (2:e vice ordförande) peter schölander (m) pia möller (m) marie tidedal (m) sandra kovács (m) lars linderot (m) bo caperman (m) boel olsson litström (m) gustaf wingårdh (m) nils hyllienmark (m) anita söderlind (m) fredrik walderyd (m) margareta widell (m) olof suneson (m) charlotte lewenhaupt (m) gertrud greén (c) pia lidwall (kd) johan ingvarson (mp) maria jönsson (mp) göran lock (mp) lennart nilsson (s) reinhold knutsson (s) carola persson (s) peter graff (s) heléne nyholm (s) alexander malmqvist (s) ann-magreth larsson (s) elisabeth eriksson (l) ros-mari paulsson (l) mikael bendz (l) christian johansson (sd) sixten paulsson (sd) lars metsäketo (sd) lars-erik nilsson (sd) johan westrell (sd) emma wennerholm (v) barbara struglics bogs (kd) ersätter fredrik eljin (kd) magnu

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Ref Files : ['1.txt']
Answer: protokoll kommunfullmäktige 2024-02-29


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

@app.route('/')
def hello():
    return 'Ready to connect to with API!'

@app.route('/Chat' , methods=['GET'])
def chat():
    response={}
    question = request.args.get('question', '')
    related_files = find_related_files(question, keywords_per_file , directory)
    content = related_files["txt"]
    answer = t5_question_answering(content, question)
    response["related_files"] = related_files["related_files"]
    answer = t5_question_answering(content, question)
    response["answer"] = answer
    return jsonify(response), 200

if __name__ == '__main__':
    # Start ngrok and create a tunnel
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)

    # Run Flask app
    app.run()


Public URL: https://5fbf-34-80-72-241.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 11:39:21] "GET /chat HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 11:39:22] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 11:39:34] "GET /chat HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 11:39:51] "GET / HTTP/1.1" 200 -
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:werkzeug:127.0.0.1 - - [09/Mar/2024 11:41:00] "GET /Chat?question=How HTTP/1.1" 200 -
